In [1]:
# Python 2 & 3 Compatibility
from __future__ import print_function, division

# Necessary imports
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
import os
import pickle
%matplotlib inline
from datetime import datetime
from dateutil.parser import parse

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
with open("all_data.pkl", 'rb') as picklefile: 
    merge_housing = pickle.load(picklefile)
merge_housing.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing,Search_Count_seattleApartment,...,New Listings Mom,New Listings Yoy,Number of Records,Price Drops,Price Drops Mom,Price Drops Yoy,Sold Above List,Sold Above List Mom,Sold Above List Yoy,Date
0,Studio,2011,03,1097.5,2011,03,382800.0,37,33,24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-03-01
1,Studio,2011,04,1095.0,2011,04,373500.0,33,22,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-04-01
2,Studio,2011,05,1025.0,2011,05,368400.0,33,25,21,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-05-01
3,Studio,2011,06,995.0,2011,06,359300.0,27,23,19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2011-06-01
4,Studio,2011,07,1000.0,2011,07,365300.0,36,25,22,...,67.50%,-16.20%,1.0,0.286394,0.082724,-0.006008,0.162996,-0.008754,0.059369,2011-07-01


In [3]:

merge_housing['target_Apartment_rent'] = merge_housing['Apartment_rent']

merge_housing = merge_housing.dropna()
merge_housing.Year = merge_housing.Year.astype(int)
merge_housing.Month = merge_housing.Month.astype(int)
merge_housing.Search_Count_SeattleRent = merge_housing.Search_Count_SeattleRent.astype(int)
merge_housing.Search_Count_SeattleHousing = merge_housing.Search_Count_SeattleHousing.astype(int)
merge_housing.Search_Count_seattleApartment = merge_housing.Search_Count_seattleApartment.astype(int)
merge_housing['Avg Sale To List Mom'] = merge_housing['Avg Sale To List Mom'].str.replace('%','').astype(float)
merge_housing['Avg Sale To List Yoy'] = merge_housing['Avg Sale To List Yoy'].str.replace('%','').astype(float)
merge_housing['New Listings Mom'] = merge_housing['New Listings Mom'].str.replace('%','').astype(float)
merge_housing['New Listings Yoy'] = merge_housing['New Listings Yoy'].str.replace('%','').astype(float)
merge_housing['Median Sale Price Mom'] = merge_housing['Median Sale Price Mom'].str.replace('%','').astype(float)
merge_housing['Median Sale Price Yoy'] = merge_housing['Median Sale Price Yoy'].str.replace('%','').astype(float)
merge_housing['Median Sale Price'] = merge_housing['Median Sale Price'].str.replace('$','').str.replace('K','000').astype(float)
merge_housing['Median List Price'] = merge_housing['Median List Price'].str.replace(',','').astype(float)
merge_housing['New Listings'] = merge_housing['New Listings'].str.replace(',','').astype(float)


merge_housing['Homes Sold'] = merge_housing['Homes Sold'].str.replace(',','').astype(int)
merge_housing['Homes Sold Mom'] = merge_housing['Homes Sold Mom'].str.replace('%','').astype(float)
merge_housing['Homes Sold Yoy'] = merge_housing['Homes Sold Yoy'].str.replace('%','').astype(float)

merge_housing['Inventory'] = merge_housing['Inventory'].str.replace(',','').astype(int)
merge_housing['Inventory Mom'] = merge_housing['Inventory Mom'].str.replace('%','').astype(float)
merge_housing['Inventory Yoy'] = merge_housing['Inventory Yoy'].str.replace('%','').astype(float)
transfer_column = {k:k.replace(' ','') for k in merge_housing.iloc[:,6:-1].columns}



merge_housing = merge_housing.rename(columns=transfer_column)



In [4]:
# Save Data to Locale

In [5]:
with open('all_clean_data.pkl', 'wb') as picklefile:
    pickle.dump(merge_housing, picklefile)

In [6]:
with open("all_clean_data.pkl", 'rb') as picklefile: 
    my_old_data = pickle.load(picklefile)
my_old_data.head()

,BedSize,Year,Month,Apartment_rent,Shift_Year,Shift_Month,Zillow_Housing_price,Search_Count_SeattleRent,Search_Count_SeattleHousing,Search_Count_seattleApartment,...,NewListingsYoy,NumberofRecords,PriceDrops,PriceDropsMom,PriceDropsYoy,SoldAboveList,SoldAboveListMom,SoldAboveListYoy,Date,target_Apartment_rent
4,Studio,2011,7,1000.0,2011,07,365300.0,36,25,22,...,-16.2,1.0,0.286394,0.082724,-0.006008,0.162996,-0.008754,0.059369,2011-07-01,1000.0
5,Studio,2011,8,1000.0,2011,08,367900.0,36,24,22,...,-11.2,1.0,0.289133,0.002739,0.019233,0.167315,0.004320,0.038875,2011-08-01,1000.0
6,Studio,2011,9,1075.0,2011,09,381900.0,38,28,22,...,-6.4,1.0,0.282218,-0.006915,-0.040034,0.255848,0.088533,0.130313,2011-09-01,1075.0
7,Studio,2011,10,995.0,2011,10,393600.0,38,29,28,...,-8.1,1.0,0.262083,-0.020135,-0.028809,0.270305,0.014457,0.104366,2011-10-01,995.0
8,Studio,2011,11,865.0,2011,11,400700.0,46,26,25,...,4.8,1.0,0.259186,-0.002897,-0.039458,0.292494,0.022190,0.136814,2011-11-01,865.0


In [7]:
my_old_data.dtypes

BedSize                                  object
Year                                      int64
Month                                     int64
Apartment_rent                          float64
Shift_Year                               object
Shift_Month                              object
Zillow_Housing_price                    float64
Search_Count_SeattleRent                  int64
Search_Count_SeattleHousing               int64
Search_Count_seattleApartment             int64
AvgSaleToList                           float64
AvgSaleToListMom                        float64
AvgSaleToListYoy                        float64
HomesSold                                 int64
HomesSoldMom                            float64
HomesSoldYoy                            float64
Inventory                                 int64
InventoryMom                            float64
InventoryYoy                            float64
MedianDom                               float64
MedianDomMom                            